In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/ec2-user/dsdemo.json'

In [3]:
from google.cloud import bigquery
client = bigquery.Client()

sql = """
SELECT year, plurality, apgar_5min, mother_age, father_age, gestation_weeks, ever_born,
case when mother_married = true then 1 else 0 end as mother_married,
weight_pounds as weight
FROM `bigquery-public-data.samples.natality`
order by rand()
limit 10000
"""

natalityDF = client.query(sql).to_dataframe().fillna(0)
natalityDF.head()

,year,plurality,apgar_5min,mother_age,father_age,gestation_weeks,ever_born,mother_married,weight
0,1975,1.0,0.0,29,31,40.0,2.0,0,9.186662
1,1982,1.0,10.0,26,26,39.0,4.0,1,6.062712
2,2006,2.0,7.0,31,34,35.0,2.0,1,5.996574
3,1991,1.0,9.0,29,31,40.0,3.0,1,8.245289
4,1979,1.0,9.0,24,31,39.0,2.0,1,8.750147


In [5]:
from sklearn.linear_model import LinearRegression
import pickle
from google.cloud import storage

model = LinearRegression()
model.fit(natalityDF.iloc[:,1:8], natalityDF['weight'])
pickle.dump(model, open('natality.pkl', 'wb'))

bucket = storage.Client().get_bucket('dsp_model_store_famenor')
blob = bucket.blob('natality/sklearn-linear')
blob.upload_from_filename('natality.pkl')